In [1]:
import os
import pyproj
import pandas as pd
import geopandas as gpd
import rasterio as rio
from datetime import datetime
from shapely.geometry import box, Polygon
import matplotlib.pyplot as plt

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
gpd.__version__

'0.13.0'

### Step 01:

    a. Load Boundary data
    b. cut the boundary geometry into 20 m x 20 m box
    

In [3]:
def subset_geom(path: str, crs: pyproj.crs.crs.CRS = 'epsg:3763', grid_resolution: int = 20):
    """generate resolution grid_res x grid_rs
    
    Parameters
    ----------
        :path
        :crs
    
    Return 
    ------
    
    """
    # Load the two GeoDataFrames
    gpd_df = gpd.read_file(path)
    original_crs = gpd_df.crs
    
    # convert degrees to meter
    gpd_df = gpd_df.to_crs(crs)

    # Get the bounding box of the shapefile
    xmin, ymin, xmax, ymax = gpd_df.total_bounds
    
    # Calculate the number of rows and columns in the grid based on the resolution
    num_cols = int((xmax - xmin) / grid_resolution)
    num_rows = int((ymax - ymin) / grid_resolution)
    
    # Generate a grid of polygons within the bounding box
    polygons = []
    for row in range(num_rows+2):
        for col in range(num_cols+2):
            x1 = xmin + col * grid_resolution
            y1 = ymin + row * grid_resolution
            x2 = x1 + grid_resolution
            y2 = y1 + grid_resolution
            polygons.append(box(x1, y1, x2, y2))
            
    # Create a GeoDataFrame from the list of polygons
    return gpd.GeoDataFrame(geometry=polygons, crs=crs).to_crs(original_crs)

In [5]:
boundary_path = '../446032/BOUNDARY1015-Rt49_Williams_E80.zip'
gdf1 = subset_geom(boundary_path)

### Step 02

    a. load yield data
    b. get geometry from overlap - intersection
    c. count each geometry area
    d. set filter to filter out the area is larger than area threshold, which is based on **area_mean - area_std**.
    

In [478]:
def clip_with_harvest(geom, yield_path, MinAreaPercentage = 0.6, TotalArea = 400.):
    """
    Argument
    --------
     :geom: boxes of geometry
     :yield_path: yield data path
     :MinAreaPercentage: an area ratio of minimum selection
    
    Return
    ------
     :normally-spaced yield data with eliminated boxes
    """
    #- Step 01: Load Yield field
    yield_df = gpd.read_file(yield_path)
    
    #- Step 02: generate a empty list to store the outputs
    valid_geom = []
    
    #- Step 03: Loop for each box geom
    #-    (a) eliminated if total area of yield data is less than 0.6 of total area
    #-    (b) spatial-weighted average
    for num, (idx, row) in enumerate(geom.iterrows()):
        #- make geometry dataframe by box
        row_gdf = gpd.GeoDataFrame(row.to_frame().T, geometry='geometry', crs = yield_df.crs)
        
        #- clipped yield in box
        clipped = gpd.clip(yield_df, row_gdf)
        clipped['clipped_area'] = clipped.to_crs('epsg:3763').area

        #- spatial-weighted average if overlapped area > 0.6 of total area
        if clipped['clipped_area'].sum() / TotalArea > MinAreaPercentage:
            valid_geom.append(spatial_normalized_average(row_gdf, clipped))

    if len(valid_geom) == 0: 
        raise ValueError({
            'status': 'failed', 
            'reason': 'no overlap, please check the inputs are correct.', 
        })
    
    valid_geom_df = pd.concat(valid_geom)
    return gpd.GeoDataFrame(valid_geom_df.reset_index().drop(columns=['index'],axis=1), geometry='geometry', crs = yield_df.crs)
        

def spatial_normalized_average(row, clipped):
    """
    Argument
    --------
       row: 20 m x 20 m box geopandas dataframe
       clipped: clipped yield dataframe
       
    Return
    ------
       normally-spaced yield data 
       
    """
    selected_clipped = clipped.select_dtypes(include=[int, float]).iloc[:,:-1]
    selected_clipped_average = selected_clipped.multiply(clipped['clipped_area'],axis='index').sum()/clipped['clipped_area'].sum()
    return selected_clipped_average.to_frame().T.assign(geometry=row['geometry'].iloc[0])

In [479]:
yield_data_path = './446032/Equipment_Harvest/2022-10-28/8BR1C4R6B/harvest/186_master_debug.zip'
valid_geometry_df = clip_with_harvest(gdf1, yield_data_path)

In [480]:
valid_geometry_df

,DISTANCE,SWATHWIDTH,BuPerAc,SECTIONID,WetMass,Moisture,Heading,Elevation,Machine,Speed,geometry
0,7.050672,30.000000,157.821588,5798.0,8837.576470,12.707058,178.502943,710.586541,1.0,4.660432,"POLYGON ((-87.93546 39.57660, -87.93560 39.576..."
1,6.114088,30.000000,220.255438,5798.0,12333.710377,12.766698,180.158402,710.740507,1.0,4.213998,"POLYGON ((-87.93560 39.57663, -87.93575 39.576..."
2,6.273544,28.807509,201.418744,5798.0,11278.958924,12.799565,188.644678,710.711547,1.0,4.250846,"POLYGON ((-87.93556 39.57675, -87.93571 39.576..."
3,6.929987,25.872022,173.129987,5798.0,9694.867724,12.654844,184.221018,710.828198,1.0,4.682281,"POLYGON ((-87.93552 39.57686, -87.93567 39.576..."
4,7.444891,29.957561,162.910097,5798.0,9122.489141,12.814833,177.789003,710.920559,1.0,5.077222,"POLYGON ((-87.93548 39.57697, -87.93562 39.577..."
...,...,...,...,...,...,...,...,...,...,...,...
1884,5.594921,21.912660,241.076427,5798.0,13499.605447,13.002778,176.029955,706.932775,1.0,3.725924,"POLYGON ((-87.94022 39.58350, -87.94036 39.583..."
1885,5.090537,29.215055,223.419001,5798.0,12510.892155,13.024157,23.595235,706.743855,1.0,3.475000,"POLYGON ((-87.94017 39.58362, -87.94032 39.583..."
1886,4.398617,29.404215,241.203209,5798.0,13506.681124,13.145848,41.602700,706.496823,1.0,3.078409,"POLYGON ((-87.94013 39.58373, -87.94028 39.583..."
1887,7.139952,30.000000,222.249451,5798.0,12445.401743,12.893960,29.302166,706.472189,1.0,4.749611,"POLYGON ((-87.94032 39.58365, -87.94047 39.583..."


### Shadow detection on Aeroptic Dataset.
 Functions

In [692]:
import rioxarray as rx
import numpy as np
def ndvi(nir, red):
    return (nir - red) / (nir + red)
    
def gndvi(nir, green):
    return (nir - green) / (nir + green)

def masked_bare_soil(red_band, green_band, images='Aeroptic'):
    masked = green_band/red_band
    threshold = 1.35 if images == 'Spot' else 1.2 if images == 'Aeroptic' else 1.15 if images == 'Pleiades' else None
    masked = green_band/red_band
    return xr.where(masked >= threshold, 1, 0)

###### test run

In [538]:
type(valid_geometry_df[['geometry']])

geopandas.geodataframe.GeoDataFrame

In [690]:

# from typing import Dict

# def image_processed(valid_geometry_df: gpd.GeoDataFrame, image: str, image_path: str, bands: Dict) -> gpd.GeoDataFrame:
#     """derive the relative values on RGB+NIR bands + NDVI + GNDVI, and average values of them
#           reference values is 10th percentile on RGB, 90th percentile on NIR, NDVI and GNDVI
          
#     Argument
#     --------
#         :valid_geometry_df, geometry of boxes
#         :image, image name
#         :image_path, data path of remote sensing images 
#         :bands, dict of RGBN, ex: {'R':1'}
        
#     Return
#     ------
#        return the original dataframe with image values
#     """
#     image_df = rx.open_rasterio(image_path)
#     masked = masked_bare_soil(
#         image_df.sel(band = bands['R']), 
#         image_df.sel(band = bands['G']), 
#         images=image
#     )
#     image_df['NDVI'] = ndvi(
#         image_df.sel(band = bands['N']), 
#         image_df.sel(band = bands['R']), 
#     )
#     image_df['GNDVI'] = ndvi(
#         image_df.sel(band = bands['N']), 
#         image_df.sel(band = bands['G']), 
#     )
    
#     image_trimmed_df = []
#     for idx, row in valid_geometry_df.iterrows():
#         row_gdf = gpd.GeoDataFrame(
#             row.to_frame().T, 
#             geometry='geometry',
#             crs = valid_geometry_df.crs
#         ).to_crs(image_df.rio.crs)

#         clipped = image_df.rio.clip(row_gdf.geometry.apply(lambda x: x.__geo_interface__), row_gdf.crs)
#         masked = masked_bare_soil(clipped.sel(band = bands['R']), clipped.sel(band = bands['G']), images=image)

#         NIR, RED = clipped.sel(band=bands['N']), clipped.sel(band=bands['R'])
#         GRN, BLU = clipped.sel(band=bands['G']), clipped.sel(band=bands['B'])

#         #- 10th percential on RGB bands
#         REDp10, GRNp10, BLUp10 = clipped.sel(band=[bands['R'], bands['G'], bands['B']]).where(masked==1).quantile(0.1, dim=['x','y']).values

#         #- 90th percential on NIR
#         NIRp90 = clipped.sel(band=[bands['N']]).where(masked==1).quantile(0.9, dim=['x','y']).values

#         #- NDVI & GNDVI, 90th percential on NDVI & GNDVI

#         NDVI = ndvi(NIR, RED)
#         GNDVI = gndvi(NIR, GRN)
#         NDVIp90 = NDVI.where(masked==1).quantile(0.9, dim=['x','y']).values
#         GNDVIp90 = GNDVI.where(masked==1).quantile(0.9, dim=['x','y']).values

#         #- Relative value 
#         RlvNIR = np.nanmean(NIR.where(masked==1).values / NIRp90)
#         RlvRED = np.nanmean(RED.where(masked==1).values / REDp10)
#         RlvGRN = np.nanmean(GRN.where(masked==1).values / GRNp10)
#         RlvBLU = np.nanmean(BLU.where(masked==1).values / BLUp10)
#         RlvNDVI = np.nanmean(NDVI.where(masked==1).values / NDVIp90)
#         RlvGNDVI = np.nanmean(GNDVI.where(masked==1).values / GNDVIp90)

#         #- Average
#         AveNIR = NIR.where(masked==1).mean().values
#         AveRED = RED.where(masked==1).mean().values
#         AveGRN = GRN.where(masked==1).mean().values
#         AveBLU = BLU.where(masked==1).mean().values
#         AveNDVI = NDVI.where(masked==1).mean().values
#         AveGNDVI = GNDVI.where(masked==1).mean().values

#         row_gdf[[f'{image} Average NIR', f'{image} Average RED', f'{image} Average Blue', 
#              f'{image} Average Green', f'{image} Average NDVI', f'{image} Average GNDVI']] = [
#         AveNIR, AveRED, AveBLU, AveGRN, AveNDVI, AveGNDVI]

#         row_gdf[[f'{image} Relative NIR', f'{image} Relative RED', f'{image} Relative Blue', 
#                  f'{image} Relative Green', f'{image} Relative NDVI', f'{image} Relative GNDVI']] = [
#             RlvNIR, RlvRED, RlvBLU, RlvGRN, RlvNDVI, RlvGNDVI]

#         image_trimmed_df.append(row_gdf)

#     image_trimmed_df = pd.concat(image_trimmed_df)
#     return gpd.GeoDataFrame(image_trimmed_df, geometry='geometry', crs = image_df.rio.crs)

In [787]:

from typing import Dict

def image_processed(valid_geometry_df: gpd.GeoDataFrame, image: str, image_path: str, bands: Dict) -> gpd.GeoDataFrame:
    """derive the relative values on RGB+NIR bands + NDVI + GNDVI, and average values of them
          reference values is 10th percentile on RGB, 90th percentile on NIR, NDVI and GNDVI
          
    Argument
    --------
        :valid_geometry_df, geometry of boxes
        :image, image name
        :image_path, data path of remote sensing images 
        :bands, dict of RGBN, ex: {'R':1'}
        
    Return
    ------
       return the original dataframe with image values
    """
    #- Step 01: Load Image
    image_df = rx.open_rasterio(image_path)

    #- Step 02: Generate a Mask using bare soil
    image_df['masked'] = masked_bare_soil(
        image_df.sel(band = bands['R']), 
        image_df.sel(band = bands['G']), 
        images=image
    )

    #- Step 03: Generate NDVI and GNDVI
    NDVI = ndvi(
        image_df.sel(band = bands['N']).where(image_df['masked'==1]),
        image_df.sel(band = bands['R']).where(image_df['masked'==1]),
    )
    image_df['NDVI'] = (('y','x'), NDVI.values)

    GNDVI = gndvi(
        image_df.sel(band = bands['N']).where(image_df['masked'==1]),
        image_df.sel(band = bands['G']).where(image_df['masked'==1]),
    )
    image_df['GNDVI'] = (('y','x'), GNDVI.values)

    #- Step 04: 10th percential on RGB bands
    REDp10 = np.nanpercentile(xr.where(image_df['masked']==1, image_df.sel(band=bands['R']), np.nan), 10)
    GRNp10 = np.nanpercentile(xr.where(image_df['masked']==1, image_df.sel(band=bands['G']), np.nan), 10)
    BLUp10 = np.nanpercentile(xr.where(image_df['masked']==1, image_df.sel(band=bands['B']), np.nan), 10)

    #- Step 05: 90th percential on NIR, NDVI, GNDVI
    NIRp90 = np.nanpercentile(xr.where(image_df['masked']==1, image_df.sel(band=bands['N']), np.nan), 90)
    NDVIp90 = np.nanpercentile(xr.where(image_df['masked']==1, image_df['NDVI'], np.nan), 90)
    GNDVIp90 = np.nanpercentile(xr.where(image_df['masked']==1, image_df['GNDVI'], np.nan), 90)

    print(f'{image}  RGBN NDVI GNDVI percentile: ', REDp10, GRNp10, BLUp10, NIRp90, NDVIp90, GNDVIp90)
    print(f'{image} GNDVI: ', GNDVI.min().values, GNDVI.max().values)
    print(f'{image}  NDVI: ',  NDVI.min().values,  NDVI.max().values, '\n\n')    
    
    #- Step 06: Clip and Generate the final outputs
    image_trimmed_df = []
    for idx, row in valid_geometry_df.iterrows():
        row_gdf = gpd.GeoDataFrame(
            row.to_frame().T, 
            geometry='geometry',
            crs = valid_geometry_df.crs
        ).to_crs(image_df.rio.crs)

        clipped = image_df.rio.clip(row_gdf.geometry.apply(lambda x: x.__geo_interface__), row_gdf.crs)

        NIR, RED = clipped.sel(band=bands['N']), clipped.sel(band=bands['R'])
        GRN, BLU = clipped.sel(band=bands['G']), clipped.sel(band=bands['B'])
        NDVI, GNDVI = clipped['NDVI'], clipped['GNDVI']

        #- Relative value in box
        RlvRED = np.nanmean(xr.where(clipped['masked']==1, RED / REDp10, np.nan))
        RlvGRN = np.nanmean(xr.where(clipped['masked']==1, GRN / GRNp10, np.nan))
        RlvBLU = np.nanmean(xr.where(clipped['masked']==1, BLU / BLUp10, np.nan))
        RlvNIR = np.nanmean(xr.where(clipped['masked']==1, NIR / NIRp90, np.nan))    
        RlvNDVI = np.nanmean(xr.where(clipped['masked']==1, NDVI / NDVIp90, np.nan))
        RlvGNDVI = np.nanmean(xr.where(clipped['masked']==1, GNDVI / GNDVIp90, np.nan))    


        #- Average in box
        AveRED = np.nanmean(xr.where(clipped['masked']==1, RED , np.nan))
        AveGRN = np.nanmean(xr.where(clipped['masked']==1, GRN , np.nan))
        AveBLU = np.nanmean(xr.where(clipped['masked']==1, BLU , np.nan))
        AveNIR = np.nanmean(xr.where(clipped['masked']==1, NIR , np.nan))    
        AveNDVI = np.nanmean(xr.where(clipped['masked']==1, NDVI , np.nan))
        AveGNDVI = np.nanmean(xr.where(clipped['masked']==1, GNDVI , np.nan))    


        row_gdf[[f'{image} Average NIR', f'{image} Average RED', f'{image} Average Blue', 
             f'{image} Average Green', f'{image} Average NDVI', f'{image} Average GNDVI']] = [
        AveNIR, AveRED, AveBLU, AveGRN, AveNDVI, AveGNDVI]

        row_gdf[[f'{image} Relative NIR', f'{image} Relative RED', f'{image} Relative Blue', 
                 f'{image} Relative Green', f'{image} Relative NDVI', f'{image} Relative GNDVI']] = [
            RlvNIR, RlvRED, RlvBLU, RlvGRN, RlvNDVI, RlvGNDVI]

        image_trimmed_df.append(row_gdf)

    image_trimmed_df = pd.concat(image_trimmed_df)
    return gpd.GeoDataFrame(image_trimmed_df, geometry='geometry', crs = image_df.rio.crs)

In [790]:
Total_now = datetime.now()

bands = ['RGBN']
bands = {char: idx+1 for idx, char in enumerate(bands[0])}
print(bands)

now = datetime.now()

image, image_path = 'Aeroptic', './446032/Aeroptic/2022-06-28/VVGUNEFVL/merged_out.tif'
image_trimmed_df = image_processed(valid_geometry_df, image, image_path, bands)
images01_cost = datetime.now() - now

print(image_trimmed_df.columns)
image_trimmed_df.head()

{'R': 1, 'G': 2, 'B': 3, 'N': 4}
Aeroptic  RGBN NDVI GNDVI percentile:  71.0 103.0 88.0 255.0 0.5644171779141104 0.4245810055865922
Aeroptic GNDVI:  0.014778325123152709 0.4868804664723032
Aeroptic  NDVI:  0.029023746701846966 0.6088328075709779 


Index(['DISTANCE', 'SWATHWIDTH', 'BuPerAc', 'SECTIONID', 'WetMass', 'Moisture',
       'Heading', 'Elevation', 'Machine', 'Speed', 'geometry',
       'Aeroptic Average NIR', 'Aeroptic Average RED', 'Aeroptic Average Blue',
       'Aeroptic Average Green', 'Aeroptic Average NDVI',
       'Aeroptic Average GNDVI', 'Aeroptic Relative NIR',
       'Aeroptic Relative RED', 'Aeroptic Relative Blue',
       'Aeroptic Relative Green', 'Aeroptic Relative NDVI',
       'Aeroptic Relative GNDVI'],
      dtype='object')


,DISTANCE,SWATHWIDTH,BuPerAc,SECTIONID,WetMass,Moisture,Heading,Elevation,Machine,Speed,...,Aeroptic Average Blue,Aeroptic Average Green,Aeroptic Average NDVI,Aeroptic Average GNDVI,Aeroptic Relative NIR,Aeroptic Relative RED,Aeroptic Relative Blue,Aeroptic Relative Green,Aeroptic Relative NDVI,Aeroptic Relative GNDVI
0,7.050672,30.0,157.821588,5798.0,8837.57647,12.707058,178.502943,710.586541,1.0,4.660432,...,64.300153,74.998469,0.521264,0.394293,0.674240,0.764510,0.730684,0.728140,0.923544,0.928664
1,6.114088,30.0,220.255438,5798.0,12333.710377,12.766698,180.158402,710.740507,1.0,4.213998,...,61.898522,71.366502,0.531207,0.406329,0.662521,0.730424,0.703392,0.692879,0.941160,0.957011
2,6.273544,28.807509,201.418744,5798.0,11278.958924,12.799565,188.644678,710.711547,1.0,4.250846,...,58.082464,67.179147,0.555149,0.416625,0.640554,0.654749,0.660028,0.652225,0.983580,0.981262
3,6.929987,25.872022,173.129987,5798.0,9694.867724,12.654844,184.221018,710.828198,1.0,4.682281,...,61.967513,74.585787,0.548065,0.395894,0.679885,0.707142,0.704176,0.724134,0.971029,0.932434
4,7.444891,29.957561,162.910097,5798.0,9122.489141,12.814833,177.789003,710.920559,1.0,5.077222,...,63.697959,77.095238,0.539969,0.393197,0.680987,0.745847,0.723840,0.748497,0.956684,0.926083


In [791]:
now = datetime.now()

image, image_path = 'Pleiades', './446032/Airbus_Pleiades/2022-06-28/YCR3AXLLG/merged_out.tif'
image_trimmed_df = image_processed(image_trimmed_df, image, image_path, bands)
images02_cost = datetime.now() - now

print(image_trimmed_df.columns)
image_trimmed_df.head()


Pleiades  RGBN NDVI GNDVI percentile:  353.0 667.0 507.0 5339.0 0.8737933500178763 0.7734438485291728
Pleiades GNDVI:  0.15726495726495726 0.7919473601754661
Pleiades  NDVI:  0.13762392875147034 0.8873092926490985 




/Users/cyhsu/miniconda3/envs/for-peter/lib/python3.10/site-packages/xarray/core/duck_array_ops.py:186: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/Users/cyhsu/miniconda3/envs/for-peter/lib/python3.10/site-packages/xarray/core/duck_array_ops.py:186: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/Users/cyhsu/miniconda3/envs/for-peter/lib/python3.10/site-packages/xarray/core/duck_array_ops.py:186: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/Users/cyhsu/miniconda3/envs/for-peter/lib/python3.10/site-packages/xarray/core/duck_array_ops.py:186: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/Users/cyhsu/miniconda3/envs/for-peter/lib/python3.10/site-packages/xarray/core/duck_array_ops.py:186: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/Users/cyhsu/miniconda3/envs/for-peter/lib/py

Index(['DISTANCE', 'SWATHWIDTH', 'BuPerAc', 'SECTIONID', 'WetMass', 'Moisture',
       'Heading', 'Elevation', 'Machine', 'Speed', 'geometry',
       'Aeroptic Average NIR', 'Aeroptic Average RED', 'Aeroptic Average Blue',
       'Aeroptic Average Green', 'Aeroptic Average NDVI',
       'Aeroptic Average GNDVI', 'Aeroptic Relative NIR',
       'Aeroptic Relative RED', 'Aeroptic Relative Blue',
       'Aeroptic Relative Green', 'Aeroptic Relative NDVI',
       'Aeroptic Relative GNDVI', 'Pleiades Average NIR',
       'Pleiades Average RED', 'Pleiades Average Blue',
       'Pleiades Average Green', 'Pleiades Average NDVI',
       'Pleiades Average GNDVI', 'Pleiades Relative NIR',
       'Pleiades Relative RED', 'Pleiades Relative Blue',
       'Pleiades Relative Green', 'Pleiades Relative NDVI',
       'Pleiades Relative GNDVI'],
      dtype='object')


,DISTANCE,SWATHWIDTH,BuPerAc,SECTIONID,WetMass,Moisture,Heading,Elevation,Machine,Speed,...,Pleiades Average Blue,Pleiades Average Green,Pleiades Average NDVI,Pleiades Average GNDVI,Pleiades Relative NIR,Pleiades Relative RED,Pleiades Relative Blue,Pleiades Relative Green,Pleiades Relative NDVI,Pleiades Relative GNDVI
0,7.050672,30.0,157.821588,5798.0,8837.57647,12.707058,178.502943,710.586541,1.0,4.660432,...,388.667131,520.913649,0.770904,0.685177,0.520380,1.028842,0.766602,0.780980,0.882250,0.885878
1,6.114088,30.0,220.255438,5798.0,12333.710377,12.766698,180.158402,710.740507,1.0,4.213998,...,334.087891,444.875977,0.839766,0.736432,0.545596,0.722698,0.658950,0.666980,0.961057,0.952147
2,6.273544,28.807509,201.418744,5798.0,11278.958924,12.799565,188.644678,710.711547,1.0,4.250846,...,328.910985,433.569129,0.842723,0.739878,0.548859,0.693112,0.648740,0.650029,0.964442,0.956602
3,6.929987,25.872022,173.129987,5798.0,9694.867724,12.654844,184.221018,710.828198,1.0,4.682281,...,340.365530,464.406250,0.826140,0.728139,0.569734,0.765149,0.671332,0.696261,0.945464,0.941425
4,7.444891,29.957561,162.910097,5798.0,9122.489141,12.814833,177.789003,710.920559,1.0,5.077222,...,389.825243,532.134709,0.811721,0.716668,0.587724,0.967329,0.768886,0.797803,0.928962,0.926594


In [792]:
now = datetime.now()

image, image_path = 'Spot', './446032/Airbus_Spot/2022-06-27/Z74PVCIAH/merged_out.tif'
image_trimmed_df = image_processed(image_trimmed_df, image, image_path, bands)
images03_cost = datetime.now() - now

print(image_trimmed_df.columns)
image_trimmed_df.head()

Spot  RGBN NDVI GNDVI percentile:  168.0 298.0 309.0 2003.0 0.8406000937646507 0.7324898328061455
Spot GNDVI:  0.19718309859154928 0.7603411513859275
Spot  NDVI:  0.2211390456644433 0.8606741573033708 




/Users/cyhsu/miniconda3/envs/for-peter/lib/python3.10/site-packages/xarray/core/duck_array_ops.py:186: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/Users/cyhsu/miniconda3/envs/for-peter/lib/python3.10/site-packages/xarray/core/duck_array_ops.py:186: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/Users/cyhsu/miniconda3/envs/for-peter/lib/python3.10/site-packages/xarray/core/duck_array_ops.py:186: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/Users/cyhsu/miniconda3/envs/for-peter/lib/python3.10/site-packages/xarray/core/duck_array_ops.py:186: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/Users/cyhsu/miniconda3/envs/for-peter/lib/python3.10/site-packages/xarray/core/duck_array_ops.py:186: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/Users/cyhsu/miniconda3/envs/for-peter/lib/py

Index(['DISTANCE', 'SWATHWIDTH', 'BuPerAc', 'SECTIONID', 'WetMass', 'Moisture',
       'Heading', 'Elevation', 'Machine', 'Speed', 'geometry',
       'Aeroptic Average NIR', 'Aeroptic Average RED', 'Aeroptic Average Blue',
       'Aeroptic Average Green', 'Aeroptic Average NDVI',
       'Aeroptic Average GNDVI', 'Aeroptic Relative NIR',
       'Aeroptic Relative RED', 'Aeroptic Relative Blue',
       'Aeroptic Relative Green', 'Aeroptic Relative NDVI',
       'Aeroptic Relative GNDVI', 'Pleiades Average NIR',
       'Pleiades Average RED', 'Pleiades Average Blue',
       'Pleiades Average Green', 'Pleiades Average NDVI',
       'Pleiades Average GNDVI', 'Pleiades Relative NIR',
       'Pleiades Relative RED', 'Pleiades Relative Blue',
       'Pleiades Relative Green', 'Pleiades Relative NDVI',
       'Pleiades Relative GNDVI', 'Spot Average NIR', 'Spot Average RED',
       'Spot Average Blue', 'Spot Average Green', 'Spot Average NDVI',
       'Spot Average GNDVI', 'Spot Relative NIR', 

,DISTANCE,SWATHWIDTH,BuPerAc,SECTIONID,WetMass,Moisture,Heading,Elevation,Machine,Speed,...,Spot Average Blue,Spot Average Green,Spot Average NDVI,Spot Average GNDVI,Spot Relative NIR,Spot Relative RED,Spot Relative Blue,Spot Relative Green,Spot Relative NDVI,Spot Relative GNDVI
0,7.050672,30.0,157.821588,5798.0,8837.57647,12.707058,178.502943,710.586541,1.0,4.660432,...,232.517241,242.344828,0.714048,0.601380,0.474487,0.975267,0.752483,0.813238,0.849450,0.821008
1,6.114088,30.0,220.255438,5798.0,12333.710377,12.766698,180.158402,710.740507,1.0,4.213998,...,226.600000,217.218182,0.789157,0.670018,0.551709,0.766613,0.733333,0.728920,0.938802,0.914713
2,6.273544,28.807509,201.418744,5798.0,11278.958924,12.799565,188.644678,710.711547,1.0,4.250846,...,201.719008,199.504132,0.782040,0.661038,0.497044,0.706218,0.652812,0.669477,0.930336,0.902453
3,6.929987,25.872022,173.129987,5798.0,9694.867724,12.654844,184.221018,710.828198,1.0,4.682281,...,247.340000,257.320000,0.768960,0.651253,0.611098,0.943571,0.800453,0.863490,0.914775,0.889095
4,7.444891,29.957561,162.910097,5798.0,9122.489141,12.814833,177.789003,710.920559,1.0,5.077222,...,254.933884,274.553719,0.735888,0.623655,0.577757,1.076692,0.825029,0.921321,0.875432,0.851417


In [793]:
type(image_trimmed_df)

geopandas.geodataframe.GeoDataFrame

In [794]:
image_trimmed_df

,DISTANCE,SWATHWIDTH,BuPerAc,SECTIONID,WetMass,Moisture,Heading,Elevation,Machine,Speed,...,Spot Average Blue,Spot Average Green,Spot Average NDVI,Spot Average GNDVI,Spot Relative NIR,Spot Relative RED,Spot Relative Blue,Spot Relative Green,Spot Relative NDVI,Spot Relative GNDVI
0,7.050672,30.0,157.821588,5798.0,8837.57647,12.707058,178.502943,710.586541,1.0,4.660432,...,232.517241,242.344828,0.714048,0.601380,0.474487,0.975267,0.752483,0.813238,0.849450,0.821008
1,6.114088,30.0,220.255438,5798.0,12333.710377,12.766698,180.158402,710.740507,1.0,4.213998,...,226.600000,217.218182,0.789157,0.670018,0.551709,0.766613,0.733333,0.728920,0.938802,0.914713
2,6.273544,28.807509,201.418744,5798.0,11278.958924,12.799565,188.644678,710.711547,1.0,4.250846,...,201.719008,199.504132,0.782040,0.661038,0.497044,0.706218,0.652812,0.669477,0.930336,0.902453
3,6.929987,25.872022,173.129987,5798.0,9694.867724,12.654844,184.221018,710.828198,1.0,4.682281,...,247.340000,257.320000,0.768960,0.651253,0.611098,0.943571,0.800453,0.863490,0.914775,0.889095
4,7.444891,29.957561,162.910097,5798.0,9122.489141,12.814833,177.789003,710.920559,1.0,5.077222,...,254.933884,274.553719,0.735888,0.623655,0.577757,1.076692,0.825029,0.921321,0.875432,0.851417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,5.594921,21.91266,241.076427,5798.0,13499.605447,13.002778,176.029955,706.932775,1.0,3.725924,...,221.927273,214.536364,0.818070,0.711757,0.643970,0.751190,0.718211,0.719921,0.973197,0.971695
1885,5.090537,29.215055,223.419001,5798.0,12510.892155,13.024157,23.595235,706.743855,1.0,3.475,...,203.909091,194.818182,0.816453,0.707879,0.572818,0.677686,0.659900,0.653752,0.971274,0.966401
1886,4.398617,29.404215,241.203209,5798.0,13506.681124,13.145848,41.6027,706.496823,1.0,3.078409,...,245.182796,239.870968,0.771158,0.665502,0.603954,0.910586,0.793472,0.804936,0.917390,0.908548
1887,7.139952,30.0,222.249451,5798.0,12445.401743,12.89396,29.302166,706.472189,1.0,4.749611,...,209.609375,208.062500,0.774692,0.678688,0.536984,0.829427,0.678347,0.698196,0.921594,0.926550


In [795]:
image_trimmed_df.to_csv('446032_processed.csv')

In [798]:
images_total_cost = datetime.now() - Total_now
images_total_cost

datetime.timedelta(seconds=387, microseconds=932097)

In [797]:
images01_cost, images02_cost, images03_cost

(datetime.timedelta(seconds=126, microseconds=285383),
 datetime.timedelta(seconds=135, microseconds=180964),
 datetime.timedelta(seconds=85, microseconds=833168))

In [799]:
(126+135+85)/60.

5.766666666666667